Heuristic analysis of BigBird-CT model generated embeddings similarities to further assess the model.

N.B. Strongly recommended to run this Notebook on a GPU

In [3]:
from pathlib import Path
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np


pd.set_option('max_colwidth', None)

# load full testing data
test_path = Path.cwd().parent.joinpath('data/interim/test_unlabelled.pkl')
test = pd.read_pickle(test_path)

# load our fine-tuned BigBird-CT with in-batch negatives model
model_bigbird_ct_path = Path.cwd().parent.joinpath('models/bigbird-ct')
model = SentenceTransformer(model_bigbird_ct_path)

sentences = test['Concatenated'].tolist()
codes = test['ModuleCode'].tolist()

# get document embeddings for our testing set modules
embeddings = model.encode(sentences,
                          batch_size = 16,
                          show_progress_bar = True)

Batches:   0%|          | 0/54 [00:00<?, ?it/s]

Attention type 'block_sparse' is not possible if sequence_length: 676 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


In addition to having measured the Spearman Rank Correlation Coefficient of this models cosine similarities with the labelled data, we will now inspect several model-generated similarities and contrast them with the actual passages of text. This serves as a further, heuristic, evaluation of the model performance.

In [4]:
# find the cosine similarity matrix for the embeddings
cos_sim = util.cos_sim(embeddings, embeddings)

# find the range of cosine similarity values
cos_sim_min = cos_sim.numpy().min()
cos_sim_max = cos_sim.numpy().max()
print(f'Range of cosine similarities: [{cos_sim_min}, {cos_sim_max}]')

Range of cosine similarities: [-0.1474636048078537, 1.0000005960464478]


We have cosine similarities ranging from approximately -0.15 to 1. What is important for a cosine similarity is where it sits relative to the range -0.15 to 1; as long as, say, cosine similarities of 0.6 are generally that between more highly semantically similar documents than documents with cosine similarities of 0.4 (and so on for other values), we can be assured our model has acceptable performance. In other words, the range of cosine similarity values are not important, their positions with respect to each other are.

We do note that our upper bound is above 1, which should not mathematically be possible for a cosine similarity. This is likely due to floating point precision and is not an issue; we can consider this upper bound, effectively, to be exactly 1. (This is easily verifiable from manually inspecting documents that share a cosine similarity of this upper bound, as they are for exactly identical documents.)

We now consider a sample of cosine similarities, specifically one comparison each of cosine similarities closest to $$\{-0.2, 0, 0.2, 0.4, 0.6, 0.8, 1\}.$$

In theory, we expect each cosine similarity to be between increasingly similar passages of text, as the cosine similarity increases. Mathematically, we might describe this as a monotonic relationship.

In [5]:
# add all pairs to a list, with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim) - 1):
    for j in range(i + 1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

# sort list in order of descending cosine similarity
all_sentence_combinations = sorted(all_sentence_combinations, key = lambda x: x[0], reverse = True)
all_sentence_combinations = np.array(all_sentence_combinations)

# get increasingly higher cosine similarity comparisons
comparisons = pd.DataFrame(columns = ['ModuleCode_A', 'Document_A', 'ModuleCode_B', 'Document_B', 'Cosine Similarity'])
for similarity in range(-1, 6):
    similarity /= 5
    index = np.absolute(all_sentence_combinations[:, 0] - similarity).argmin()
    score, i, j = all_sentence_combinations[index]
    i = int(i)
    j = int(j)
    comparisons.loc[len(comparisons)] = [codes[i], sentences[i], codes[j], sentences[j], float(cos_sim[i][j])]

comparisons

ModuleCode_A  \
0    [HIS0002]   
1    [MCH3074]   
2    [TCP8902]   
3    [NBS8615]   
4    [SPE1051]   
5    [MUS8017]   
6    [HSS3220]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

We begin by writing some comments about the similarities between the documents described within each cosine similarity rating.

| Cosine Similarity | Comments                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|-------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| -0.147464         | Document A is very short, referring to a dummy module. Document B is much longer, and refers to a very specific module, one that is specifically for the BA (Hons) Geography programmes.<br>Document B refers to many specific concepts, such as data analysis, ethics of research and human geography, whereas Document A is very vague.<br>These documents are highly dissimilar.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| -0.000044         | Document A is quite long, referring to concepts within media, communication and culture. Document B is very short, referring to a dummy module, interestingly the same one as in cosine similarity -0.147464.<br>Document A refers to many specific concepts, such as cultural studies, accounts of culture and historical developments, whereas Document B is very vague.<br>These documents are highly dissimilar.                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| 0.200004          | Document A discusses ethics within planning practice. Document B refers to organic chemistry.<br>Document A mentions more general concepts and questions, whereas Document B refers to specific concepts within organic chemistry (e.g. nucleophilic addition).<br>These documents are dissimilar as the documents refer to different topics, Document A describes more general ethical themes, and Document B refers to specific scientific concepts.                                                                                                                                                                                                                                                                                                                                                                                                                |
| 0.400000          | Document A refers to experimental economics. Document B refers to hydrogen use within the industrial sector.<br>Document A discusses specific scientific concepts within economics, such as market equilibrium, assessing experimental results and assess experimental literature. Document B refers to how hydrogen is produced and used as fuel, mentioning physical chemistry, reactor engineer, heat transfer, among many others.<br>These documents are generally different, but have their similarities as they both discuss domain-specific scientific methods within their respective disciplines.                                                                                                                                                                                                                                                            |
| 0.599999          | Document A refers to research methods and questions within the clinical context. Document B refers to practical laboratory and study skills, in the biomedical/biomolecular context.<br>Document A is generally on the subject of understanding experimental design and introducing statistics, and mentions that this is for the purpose of applying these methods to psychology, speech language pathology and child development. Document B generally describes developing analytical techniques, data understanding and practical skills, with reference of applying this to spectrophotometry, ion exchange chromatography, enzyme kinetics, among other areas.<br>These documents are related as they both describe learning general scientific and quantitative skills to be applied to their respective domains, with these respective domains being related. |
| 0.800044          | Document A discusses postgraduate students choosing a specialist musical practice for practical study. Document B discusses postgraduate students developing their own creative musical composition.<br>Both documents are within the musical context, refer to scheduled workshops, working with visiting experts, among others. Their key differences is that one deals with musical practice, and the other with developing a musical composition.<br>These documents are highly related as they both refer to the same general area, module and learning structure, but focusing on different techniques within music.                                                                                                                                                                                                                                            |
| 1.000000          | Document A and Document B are both very short, mentioning that they are dummy modules.<br>Document A and Document B are both identical.<br>These documents are semantically, and indeed lexicographically, identical.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |

Hence, looking each cosine similarity, we see that as the cosine similarity increases, the documents within the pairings are increasingly semantically similar. By this, it's reasonable to say that our modelling pipeline has generated document embeddings representative of the semantic content of the documents within our corpus.

The only exception to this could arguably be the documents within cosine similarities -0.147464 and -0.000044, with them both being roughly arguably as dissimilar to each other, however this doesn't mean that the documents within similarity -0.000044 are more dissimilar than those within similarity -0.147464 and thus the monotonic relationship is not broken.

We now do the same as the above, but instead looking at cosine similarities closest to $$\{-0.1, 0.1, 0.3, 0.5, 0.7, 0.9\},$$ to serve as further investigation into the relationship between the evaluated cosine similarities and semantic similarity of the documents.

In [6]:
# get increasingly higher cosine similarity comparisons, for the alternative set of similarities
comparisons_2 = pd.DataFrame(columns = ['ModuleCode_A', 'Document_A', 'ModuleCode_B', 'Document_B', 'Cosine Similarity'])
for similarity in range(-1, 5):
    similarity = similarity / 5 + 0.1
    index = np.absolute(all_sentence_combinations[:, 0] - similarity).argmin()
    score, i, j = all_sentence_combinations[index]
    i = int(i)
    j = int(j)
    comparisons_2.loc[len(comparisons_2)] = [codes[i], sentences[i], codes[j], sentences[j], float(cos_sim[i][j])]

comparisons_2

ModuleCode_A  \
0           [HIS0001]   
1           [TMP9001]   
2           [ACC2025]   
3           [CME3040]   
4  [BUS2074, LBU2074]   
5           [EEE2012]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

We omit commenting about the documents featured in the comparisons here, but again we see an apparently monotonic relationship between the semantic similarity of the documents and the displayed cosine similarities.